# 炼油厂优化

## 目标和准备工作

在这个示例中，我们将演示如何使用数学优化来优化炼油厂的产出。你将学习如何生成一个最大化总利润的最优生产计划，同时考虑生产能力和其他限制条件。

关于这类模型的更多信息，可以在 H. P. Williams 所著《Modeling Building in Mathematical Programming》第五版的第258页和第306-310页的示例#6中找到。

这是一个中级水平的建模示例，我们假设你了解 Python 并熟悉 Gurobi Python API。此外，你还应该对构建数学优化模型有一定的了解。

**下载代码库** <br />
你可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一个炼油厂购买两种原油(原油1和原油2)，并通过四个步骤(蒸馏、重整、裂化和混合)将其提炼成可销售的汽油和燃料。

### 第一步：蒸馏

蒸馏过程根据沸点将每种原油分离成六种馏分：轻石脑油、中石脑油、重石脑油、轻油、重油和残渣油。轻、中、重石脑油的辛烷值分别为90、80和70。

一桶原油分离成以下馏分：

| <i></i> | 轻石脑油 | 中石脑油 | 重石脑油 | 轻油 | 重油 | 残渣油 |
| --- | --- | --- | --- | --- | --- | --- |
| 原油1 | 0.1 | 0.2 | 0.2 | 0.12 | 0.2 | 0.13 |
| 原油2 | 0.15 | 0.25 | 0.18 | 0.08 | 0.19 | 0.12 |

蒸馏过程中有少量损耗(原油1为5%，原油2为3%)。

### 第二步：重整

蒸馏后，得到的石脑油可以混合成不同等级的汽油，或者通过重整过程处理。重整过程的产物是辛烷值为115的重整汽油。

不同石脑油每桶的重整汽油产量如下：

- 一桶轻石脑油可产出0.6桶重整汽油
- 一桶中石脑油可产出0.52桶重整汽油
- 一桶重石脑油可产出0.45桶重整汽油

### 第三步：裂化

轻油和重油可以混合成喷气燃料，或者通过催化裂化处理。催化裂化装置生产裂化油和裂化汽油。

裂化汽油的辛烷值为105，产量如下：

- 一桶轻油可产出0.68桶裂化油和0.28桶裂化汽油
- 一桶重油可产出0.75桶裂化油和0.2桶裂化汽油

裂化油用于混合燃料油和喷气燃料；裂化汽油用于混合汽油。残渣油可用于生产润滑油或混合成喷气燃料和燃料油：

- 一桶残渣油可产出0.5桶润滑油

### 第四步：混合

#### 汽油

通过混合石脑油、重整汽油和裂化汽油可以生产两种汽油 — 普通汽油和高级汽油。唯一的要求是普通汽油的辛烷值至少为84，高级汽油的辛烷值至少为94。假设辛烷值按体积线性混合。

#### 喷气燃料

喷气燃料的蒸气压不得超过1.0 kg/cm²。轻油、重油、裂化油和残渣油的蒸气压分别为1.0、0.6、1.5和0.05 kg/cm²。假设蒸气压按体积线性混合。

#### 燃料油

生产燃料油时，必须按10:4:3:1的比例混合轻油、裂化油、重油和残渣油。

可用性和产能限制如下：

- 原油1的日可用量为20,000桶
- 原油2的日可用量为30,000桶
- 每日最多可蒸馏45,000桶原油
- 每日最多可重整10,000桶石脑油
- 每日最多可裂化8,000桶油料
- 每日润滑油产量必须在500到1,000桶之间
- 高级汽油产量必须至少为普通汽油产量的40%

每种最终产品的每桶利润贡献(单位：美分)如下：

| <i></i> | 利润贡献 |
| --- | --- |
| 高级汽油 | 700 |
| 普通汽油 | 600 |
| 喷气燃料 | 400 |
| 燃料油 | 350 |
| 润滑油 | 150 |

关键问题是：如何规划炼油厂的运营以最大化总利润？

---
## 模型公式

### 集合和索引

$i \in \text{Crudes}=\{1,2\}$: 原油集合

### 参数

$\text{buy_limit}_i \in \mathbb{N}$: 原油 $i$ 的最大购买量(桶)

$\text{distill_cap} \in \mathbb{N}$: 最大蒸馏原油量(桶)

$\text{reform_cap} \in \mathbb{N}$: 最大重整石脑油量(桶)

$\text{crack_cap} \in \mathbb{N}$ 最大裂化油料量(桶)

$\text{LBO_min}, \text{LBO_max} \in \mathbb{N}$ 润滑油生产的最小和最大量(桶)

### 决策变量

$\text{CR}_i \in [0,\text{buy_limit}_i] \subset \mathbb{R}^+$: 购买的原油 $i$ 的数量(桶)

$\text{LN} \in \mathbb{R}^+$: 蒸馏的轻石脑油数量(桶)

$\text{MN} \in \mathbb{R}^+$: 蒸馏的中石脑油数量(桶)

$\text{HN} \in \mathbb{R}^+$: 蒸馏的重石脑油数量(桶)

$\text{LO} \in \mathbb{R}^+$: 蒸馏的轻油数量(桶)

$\text{HO} \in \mathbb{R}^+$: 蒸馏的重油数量(桶)

$\text{R} \in \mathbb{R}^+$: 蒸馏的残渣油数量(桶)

$\text{LNRG} \in \mathbb{R}^+$: 用于生产重整汽油的轻石脑油数量(桶)

$\text{MNRG} \in \mathbb{R}^+$: 用于生产重整汽油的中石脑油数量(桶)

$\text{HNRG} \in \mathbb{R}^+$: 用于生产重整汽油的重石脑油数量(桶)

$\text{RG} \in \mathbb{R}^+$: 生产的重整汽油数量(桶)

$\text{LOCGO} \in \mathbb{R}^+$: 用于生产裂化汽油和裂化油的轻油数量(桶)

$\text{HOCGO} \in \mathbb{R}^+$: 用于生产裂化汽油和裂化油的重油数量(桶)

$\text{CG} \in \mathbb{R}^+$: 生产的裂化汽油数量(桶)

$\text{CO} \in \mathbb{R}^+$: 生产的裂化油数量(桶)

$\text{LNPMF} \in \mathbb{R}^+$: 用于生产高级汽油的轻石脑油数量(桶)

$\text{LNRMF} \in \mathbb{R}^+$: 用于生产普通汽油的轻石脑油数量(桶)

$\text{MNPMF} \in \mathbb{R}^+$: 用于生产高级汽油的中石脑油数量(桶)

$\text{MNRMF} \in \mathbb{R}^+$: 用于生产普通汽油的中石脑油数量(桶)

$\text{HNPMF} \in \mathbb{R}^+$: 用于生产高级汽油的重石脑油数量(桶)

$\text{HNRMF} \in \mathbb{R}^+$: 用于生产普通汽油的重石脑油数量(桶)

$\text{RGPMF} \in \mathbb{R}^+$: 用于生产高级汽油的重整汽油数量(桶)

$\text{RGRMF} \in \mathbb{R}^+$: 用于生产普通汽油的重整汽油数量(桶)

$\text{CGPMF} \in \mathbb{R}^+$: 用于生产高级汽油的裂化汽油数量(桶)

$\text{CGRMF} \in \mathbb{R}^+$: 用于生产普通汽油的裂化汽油数量(桶)

$\text{LOJF} \in \mathbb{R}^+$: 用于生产喷气燃料的轻油数量(桶)

$\text{HOJF} \in \mathbb{R}^+$: 用于生产喷气燃料的重油数量(桶)

$\text{RJF} \in \mathbb{R}^+$: 用于生产喷气燃料的残渣油数量(桶)

$\text{COJF} \in \mathbb{R}^+$: 用于生产喷气燃料的裂化油数量(桶)

$\text{RLBO} \in \mathbb{R}^+$: 用于生产润滑油的残渣油数量(桶)

$\text{PMF} \in \mathbb{R}^+$: 生产的高级汽油数量(桶)

$\text{RMF} \in \mathbb{R}^+$: 生产的普通汽油数量(桶)

$\text{JF} \in \mathbb{R}^+$: 生产的喷气燃料数量(桶)

$\text{FO} \in \mathbb{R}^+$: 生产的燃料油数量(桶)

$\text{LBO} \in [\text{LBO_min}, \text{LBO_max}] \subset \mathbb{R}^+$: 生产的润滑油数量(桶)

### 目标函数

- **利润:** 最大化总利润(单位：百美元)

\begin{equation}
\text{Max} \quad Z = 7*\text{PMF} + 6*\text{RMF} + 4*\text{JF} + 3.5*\text{FO} + 1.5*\text{LBO}
\tag{0}
\end{equation}

### 约束条件

- **蒸馏能力:** 蒸馏的原油数量不能超过产能

\begin{equation}
\sum_{i \in \text{Crudes}}{\text{CR}_i} \leq \text{distill_cap}
\tag{1}
\end{equation}

- **重整能力:** 重整的石脑油数量不能超过产能

\begin{equation}
\text{LNRG} + \text{MNRG} + \text{HNRG} \leq \text{reform_cap}
\tag{2}
\end{equation}

- **裂化能力:** 裂化的油料数量不能超过产能

\begin{equation}
\text{LOCGO} + \text{HOCGO} \leq \text{crack_cap}
\tag{3}
\end{equation}

- **产量:** 生产的数量取决于使用的投入量及其相应的产量

\begin{equation}
0.10*\text{CR}_1 + 0.15*\text{CR}_2 = \text{LN}
\tag{4.1}
\end{equation}

\begin{equation}
0.20*\text{CR}_1 + 0.25*\text{CR}_2 = \text{MN}
\tag{4.2}
\end{equation}

\begin{equation}
0.20*\text{CR}_1 + 0.18*\text{CR}_2 = \text{HN}
\tag{4.3}
\end{equation}

\begin{equation}
0.12*\text{CR}_1 + 0.08*\text{CR}_2 = \text{LO}
\tag{4.4}
\end{equation}

\begin{equation}
0.20*\text{CR}_1 + 0.19*\text{CR}_2 = \text{HO}
\tag{4.5}
\end{equation}

\begin{equation}
0.13*\text{CR}_1 + 0.12*\text{CR}_2 = \text{R}
\tag{4.6}
\end{equation}

\begin{equation}
0.60*\text{LNRG} + 0.52*\text{MNRG} + 0.45*\text{HNRG} = \text{RG}
\tag{4.7}
\end{equation}

\begin{equation}
0.68*\text{LOCGO} + 0.75*\text{HOCGO} = \text{CO}
\tag{4.8}
\end{equation}

\begin{equation}
0.28*\text{LOCGO} + 0.20*\text{HOCGO} = \text{CG}
\tag{4.9}
\end{equation}

\begin{equation}
0.50*\text{RLBO} = \text{LBO}
\tag{4.10}
\end{equation}

\begin{equation}
\text{LNPMF} + \text{MNPMF} + \text{HNPMF} + \text{RGPMF} + \text{CGPMF} = \text{PMF}
\tag{4.11}
\end{equation}

\begin{equation}
\text{LNRMF} + \text{MNRMF} + \text{HNRMF} + \text{RGRMF} + \text{CGRMF} = \text{RMF}
\tag{4.12}
\end{equation}

\begin{equation}
\text{LOJF} + \text{HOJF} + \text{COJF} + \text{RJF} = \text{JF}
\tag{4.13}
\end{equation}

- **质量守恒:** 使用的数量必须等于可用的数量

\begin{equation}
\text{LNRG} + \text{LNPMF} + \text{LNRMF} = \text{LN}
\tag{5.1}
\end{equation}

\begin{equation}
\text{MNRG} + \text{MNPMF} + \text{MNRMF} = \text{MN}
\tag{5.2}
\end{equation}

\begin{equation}
\text{HNRG} + \text{HNPMF} + \text{HNRMF} = \text{HN}
\tag{5.3}
\end{equation}

\begin{equation}
\text{LOCGO} + \text{LOJF} + 0.55*\text{FO} = \text{LO}
\tag{5.4}
\end{equation}

\begin{equation}
\text{HOCGO} + \text{HOJF} + 0.17*\text{FO} = \text{HO}
\tag{5.5}
\end{equation}

\begin{equation}
\text{COJF} + 0.22*\text{FO} = \text{CO}
\tag{5.6}
\end{equation}

\begin{equation}
\text{RLBO} + \text{RJF} + 0.0555*\text{FO} = \text{R}
\tag{5.7}
\end{equation}

\begin{equation}
\text{CGPMF} + \text{CGRMF} = \text{CG}
\tag{5.8}
\end{equation}

\begin{equation}
\text{RGPMF} + \text{RGRMF} = \text{RG}
\tag{5.9}
\end{equation}

- **高级汽油与普通汽油的比例:** 高级汽油与普通汽油的生产比例必须满足最低要求

\begin{equation}
\text{PMF} \geq 0.40*\text{RMF}
\tag{6}
\end{equation}

- **辛烷值容差:** 每种汽油的辛烷值不能低于下限

\begin{equation}
90*\text{LNPMF} + 80*\text{MNPMF} + 70*\text{HNPMF} + 115*\text{RGPMF} + 105*\text{CGPMF} \geq 94*\text{PMF}
\tag{7.1}
\end{equation}

\begin{equation}
90*\text{LNRMF} + 80*\text{MNRMF} + 70*\text{HNRMF} + 115*\text{RGRMF} + 105*\text{CGRMF} \geq 84*\text{PMF}
\tag{7.2}
\end{equation}

- **蒸气压容差:** 喷气燃料的蒸气压不能低于下限

\begin{equation}
1.0*\text{LOJF} + 0.6*\text{HOJF} + 1.5*\text{COJF} + 0.05*\text{RJF} \leq 1.0*\text{JF}
\tag{8}
\end{equation}

---
## Python 实现

我们导入 Gurobi Python 模块和其他 Python 库。

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [ ]:
# 参数

crude_numbers = range(1,2+1)
petrols = ["Premium_fuel", "Regular_fuel"]
end_product_names = ["Premium_fuel", "Regular_fuel", "Jet_fuel", "Fuel_oil", "Lube_oil"]
distillation_products_names = ["Light_naphtha", "Medium_naphtha", "Heavy_naphtha",
                               "Light_oil", "Heavy_oil", "Residuum"]
naphthas = ["Light_naphtha", "Medium_naphtha", "Heavy_naphtha"]
intermediate_oils = ["Light_oil", "Heavy_oil"]
cracking_products_names = ["Cracked_gasoline", "Cracked_oil"]
used_for_motor_fuel_names = ["Light_naphtha", "Medium_naphtha", "Heavy_naphtha",
                             "Reformed_gasoline", "Cracked_gasoline"]
used_for_jet_fuel_names = ["Light_oil", "Heavy_oil", "Residuum", "Cracked_oil"]

buy_limit = {1:20000, 2:30000}
lbo_min = 500
lbo_max = 1000

distill_cap = 45000
reform_cap = 10000
crack_cap = 8000

distillation_splitting_coefficients = {"Light_naphtha": (0.1, 0.15),
                          "Medium_naphtha": (0.2, 0.25),
                         "Heavy_naphtha": (0.2, 0.18),
                         "Light_oil": (0.12, 0.08),
                         "Heavy_oil": (0.2, 0.19),
                         "Residuum": (0.13, 0.12)}

cracking_splitting_coefficients = {("Light_oil","Cracked_oil"): 0.68,
                                   ("Heavy_oil","Cracked_oil"): 0.75,
                                   ("Light_oil","Cracked_gasoline"): 0.28,
                                   ("Heavy_oil","Cracked_gasoline"): 0.2}

reforming_splitting_coefficients = {"Light_naphtha": 0.6, "Medium_naphtha":0.52, "Heavy_naphtha":0.45}
end_product_profit = {"Premium_fuel":7, "Regular_fuel":6, "Jet_fuel":4, "Fuel_oil":3.5, "Lube_oil":1.5}
blending_coefficients = {"Light_oil": 0.55, "Heavy_oil": 0.17, "Cracked_oil": 0.22, "Residuum": 0.055}

lube_oil_factor = 0.5
pmf_rmf_ratio = 0.4

octance_number_coefficients = {
    "Light_naphtha":90,
    "Medium_naphtha":80,
    "Heavy_naphtha":70,
    "Reformed_gasoline":115,
    "Cracked_gasoline":105,
}
octance_number_fuel = {"Premium_fuel": 94,"Regular_fuel": 84}

vapor_pressure_constants = [0.6, 1.5, 0.05]

## 模型部署
我们创建一个模型和变量。

In [3]:
refinery = gp.Model('Refinery_Optimization')

# Variables
crudes = refinery.addVars(crude_numbers, ub=buy_limit, name="cr")    
end_products = refinery.addVars(end_product_names, name="end_prod")
end_products["Lube_oil"].lb= lbo_min
end_products["Lube_oil"].ub= lbo_max
distillation_products = refinery.addVars(distillation_products_names, name="dist_prod")
reform_usage = refinery.addVars(naphthas, name="napthas_to_reformed_gasoline")
reformed_gasoline = refinery.addVar(name="reformed_gasoline")
cracking_usage = refinery.addVars(intermediate_oils,name="intermediate_oils_to_cracked_gasoline")
cracking_products = refinery.addVars(cracking_products_names,  name="cracking_prods")
used_for_regular_motor_fuel = refinery.addVars(used_for_motor_fuel_names, name="motor_fuel_to_regular_motor_fuel")
used_for_premium_motor_fuel = refinery.addVars(used_for_motor_fuel_names, name="motot_fuel_to_premium_motor_fuel")
used_for_jet_fuel = refinery.addVars(used_for_jet_fuel_names, name="jet_fuel")
used_for_lube_oil = refinery.addVar(vtype=GRB.CONTINUOUS,name="residuum_used_for_lube_oil")

Using license file c:\gurobi\gurobi.lic


接下来，我们插入约束条件。

蒸馏能力约束为：

In [ ]:
# 1. 蒸馏能力
DistillationCap = refinery.addConstr(crudes.sum() <= distill_cap, "Distill_cap")

重整能力约束为：

In [ ]:
# 2. 重整能力
ReformingCap = refinery.addConstr(reform_usage.sum() <= reform_cap, "Reform_cap")

裂化能力约束为：

In [ ]:
# 3. 裂化能力
CrackingCap = refinery.addConstr(cracking_usage.sum() <= crack_cap, "Crack_cap")

蒸馏产品的数量取决于使用的原油数量，并考虑到每种原油在蒸馏过程中的分离方式。这给出了：

In [ ]:
# 4.1-4.6 产量 (原油产品)
YieldCrudeOil = refinery.addConstrs((gp.quicksum(distillation_splitting_coefficients[dpn][crude-1]*crudes[crude] for crude in crudes)
                  == distillation_products[dpn] for dpn in distillation_products_names), "Splitting_distillation")

重整汽油的数量取决于重整过程中使用的石脑油数量。这给出了约束条件：

In [ ]:
# 4.7 产量 (石脑油重整)
YieldNaphthas = refinery.addConstr(reform_usage.prod(reforming_splitting_coefficients) == reformed_gasoline, "Splitting_reforming")

裂化油和裂化汽油的数量取决于使用的轻油和重油数量。这给出了约束条件：

In [ ]:
# 4.8-4.9 产量 (油料裂化)
YieldCrackingOil = refinery.addConstrs((gp.quicksum(cracking_splitting_coefficients[oil, crack_prod]*cracking_usage[oil]
                           for oil in intermediate_oils) == cracking_products[crack_prod]
                  for crack_prod in cracking_products_names),
                 name="Splitting_cracking")

生产的润滑油数量是使用的残渣油数量的0.5倍。这给出了：

In [ ]:
# 4.10 产量 (润滑油)
YieldLubeOil = refinery.addConstr(lube_oil_factor*used_for_lube_oil == end_products["Lube_oil"],
                "Splitting_lube_oil")

生产的汽油和喷气燃料的数量等于其成分的总数量。这给出了约束条件：

In [ ]:
# 4.11 产量 (高级汽油)
YieldPremium = refinery.addConstr(used_for_premium_motor_fuel.sum() == end_products["Premium_fuel"], "Blending_premium_fuel")

# 4.12 产量 (普通汽油)
YieldRegular = refinery.addConstr(used_for_regular_motor_fuel.sum() == end_products["Regular_fuel"], "Blending_regular_fuel")

# 4.13 产量 (喷气燃料)
YieldJetFuel = refinery.addConstr(used_for_jet_fuel.sum() == end_products["Jet_fuel"], "Continuity_jet_fuel")

用于重整和混合的石脑油数量等于可用的数量。这给出了：

In [ ]:
# 5.1-5.3 质量守恒 (石脑油)
MassBalNaphthas = refinery.addConstrs((reform_usage[naphtha] +
                    used_for_regular_motor_fuel[naphtha] +
                    used_for_premium_motor_fuel[naphtha] ==
                    distillation_products[naphtha] for naphtha in naphthas), "Continuity_napththa")

对于燃料油的混合，轻油/重油/裂化油/残渣油的比例是固定的。因此，没有为这个比例引入单独的变量，因为它由变量决定。这给出了：

In [ ]:
# 5.4 质量守恒 (轻油)
MassBalLightOil = refinery.addConstr(cracking_usage["Light_oil"]+
                used_for_jet_fuel["Light_oil"]+
                blending_coefficients["Light_oil"]*end_products["Fuel_oil"] ==
                distillation_products["Light_oil"], "Fixed_proportion_light_oil_for_blending")

# 5.5 质量守恒 (重油)
MassBalHeavyOil = refinery.addConstr(cracking_usage["Heavy_oil"]+
                used_for_jet_fuel["Heavy_oil"]+
                blending_coefficients["Heavy_oil"]*end_products["Fuel_oil"] ==
                distillation_products["Heavy_oil"], "Fixed_proportion_heavy_oil_for_blending")

# 5.6 质量守恒 (裂化油)
MassBalCrackedOil = refinery.addConstr(used_for_jet_fuel["Cracked_oil"]+
                blending_coefficients["Cracked_oil"]*end_products["Fuel_oil"] ==
                cracking_products["Cracked_oil"], "Fixed_proportion_cracked_oil_for_blending")

# 5.7 质量守恒 (残渣油)
MassBalResiduum = refinery.addConstr(used_for_lube_oil +
                used_for_jet_fuel["Residuum"]+
                blending_coefficients["Residuum"]*end_products["Fuel_oil"] ==
                distillation_products["Residuum"], "Fixed_proportion_residuum_for_blending")

# 5.8 质量守恒 (裂化汽油)
MassBalCrackedGas = refinery.addConstr(used_for_regular_motor_fuel["Cracked_gasoline"] +
                used_for_premium_motor_fuel["Cracked_gasoline"] ==
                cracking_products["Cracked_gasoline"], "Continuity_cracked_gasoline")

# 5.9 质量守恒 (重整汽油)
MassBalReformedGas = refinery.addConstr(used_for_regular_motor_fuel["Reformed_gasoline"] +
                used_for_premium_motor_fuel["Reformed_gasoline"] ==
                reformed_gasoline, "Continuity_reformed_gasoline")

高级汽油的生产必须至少是普通汽油生产的40%，这给出了：

In [ ]:
# 7. 高级汽油与普通汽油的比例
Premium2Regular = refinery.addConstr(end_products["Premium_fuel"] >= pmf_rmf_ratio*end_products["Regular_fuel"],
                "Prem2reg_prop")

有必要规定高级汽油(普通汽油)的辛烷值不低于94(84)。这由约束条件给出：

In [ ]:
# 8.1-8.2 辛烷值容差
OctaneRegular = refinery.addConstr(used_for_regular_motor_fuel.prod(octance_number_coefficients) >=
                octance_number_fuel["Regular_fuel"] * end_products["Regular_fuel"],
                "Octane_tol_regular_fuel")

OctanePremium = refinery.addConstr(used_for_premium_motor_fuel.prod(octance_number_coefficients) >=
                octance_number_fuel["Premium_fuel"] * end_products["Premium_fuel"],
                "Octane_tol_premium_fuel")

对于喷气燃料，我们有蒸气压的约束条件：

In [ ]:
# 9. 蒸气压容差
VaporPressure = refinery.addConstr(used_for_jet_fuel["Light_oil"] +
                vapor_pressure_constants[0]*used_for_jet_fuel["Heavy_oil"] +
                vapor_pressure_constants[1]*used_for_jet_fuel["Cracked_oil"] +
                vapor_pressure_constants[2]*used_for_jet_fuel["Residuum"] <= end_products["Jet_fuel"],
                "Vapor_pressure_tol")

该模型有29个约束条件以及三个变量的简单界限。

关于燃料油的混合应注意一点，其中的成分(轻油、重油、裂化油和残渣油)按固定比例使用。可能更适合将燃料油的生产视为一种活动。在石油工业中，通常以活动的形式思考，而不是数量。在 H.P. Williams 的书的第3.4节中，讨论了将活动表示为过程的极端操作模式的模型公式。在这个示例中，我们有一个特殊的过程，只有一种操作模式。这个活动的水平自动确定了成分的比例。

唯一涉及利润(或成本)的变量是最终产品。这给出了一个要最大化的目标(单位：美元)：

In [ ]:
# 0. 利润
refinery.setObjective(end_products.prod(end_product_profit), GRB.MAXIMIZE)

接下来，优化过程开始，Gurobi 找到最优解。

In [18]:
refinery.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 29 rows, 36 columns and 106 nonzeros
Model fingerprint: 0xe30699e6
Coefficient statistics:
  Matrix range     [5e-02, 1e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [5e+02, 3e+04]
  RHS range        [8e+03, 5e+04]
Presolve removed 13 rows and 14 columns
Presolve time: 0.01s
Presolved: 16 rows, 22 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1887574e+06   6.045565e+04   0.000000e+00      0s
      14    2.1136513e+05   0.000000e+00   0.000000e+00      0s

Solved in 14 iterations and 0.01 seconds
Optimal objective  2.113651348e+05


---
## 分析

最优解的利润为 $\$211,365.13$。变量的最优值如下：

In [19]:
for var in refinery.getVars():
    if abs(var.x) > 1e-6:
        print("{0} = {1}".format(var.varName, np.round(var.x, 2)))

cr[1] = 15000.0
cr[2] = 30000.0
end_prod[Premium_fuel] = 6817.78
end_prod[Regular_fuel] = 17044.45
end_prod[Jet_fuel] = 15156.0
end_prod[Lube_oil] = 500.0
dist_prod[Light_naphtha] = 6000.0
dist_prod[Medium_naphtha] = 10500.0
dist_prod[Heavy_naphtha] = 8400.0
dist_prod[Light_oil] = 4200.0
dist_prod[Heavy_oil] = 8700.0
dist_prod[Residuum] = 5550.0
napthas_to_reformed_gasoline[Heavy_naphtha] = 5406.86
reformed_gasoline = 2433.09
intermediate_oils_to_cracked_gasoline[Light_oil] = 4200.0
intermediate_oils_to_cracked_gasoline[Heavy_oil] = 3800.0
cracking_prods[Cracked_gasoline] = 1936.0
cracking_prods[Cracked_oil] = 5706.0
motor_fuel_to_regular_motor_fuel[Light_naphtha] = 273.07
motor_fuel_to_regular_motor_fuel[Medium_naphtha] = 10500.0
motor_fuel_to_regular_motor_fuel[Heavy_naphtha] = 2993.14
motor_fuel_to_regular_motor_fuel[Reformed_gasoline] = 1342.24
motor_fuel_to_regular_motor_fuel[Cracked_gasoline] = 1936.0
motot_fuel_to_premium_motor_fuel[Light_naphtha] = 5726.93
motot_fuel_to_premium

---
## 参考文献

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC